## CESM2 - LARGE ENSEMBLE (LENS2)

#### by Mauricio Rocha and Dr. Gustavo Marques

 - This Notebook aims to find the maximum AMOC in the North and South Atlantic, and the minimum AMOC in the South Atlantic.

## Imports

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np 
import dask
import cf_xarray
import intake
import cftime
import nc_time_axis
import intake_esm
import matplotlib.pyplot as plt
import pop_tools
from dask.distributed import Client
from ncar_jobqueue import NCARCluster
import warnings, getpass, os

## Improve the workflow using clusters 

In [ ]:
mem_per_worker = 3 # in GB 
num_workers = 60 
cluster = NCARCluster(cores=1, processes=3, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=6:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Read in OGCM history file & MOC template file

In [ ]:
catalog = intake.open_esm_datastore(
    '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
)

In [ ]:
cat_subset = catalog.search(component='ocn',variable=['MOC'],frequency='month_1',experiment="historical")
del catalog

In [ ]:
# Load catalog entries for subset into a dictionary of xarray datasets
dset_dict_raw  = cat_subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
print(f"\nDataset dictionary keys:\n {dset_dict_raw.keys()}")
del cat_subset

In [ ]:
%%time
fb=('cmip6','smbb')
for ifb in range(0,2):  
    print(f"Forcing_variant: {ifb}")
    str=f'ds_{fb[ifb]}_moc = dset_dict_raw[\'ocn.historical.pop.h.{fb[ifb]}.MOC\']' 
    exec(str)
    str=f'ds_{fb[ifb]}_moc = ds_{fb[ifb]}_moc.sel(time=slice(\'1850-01-01\', \'2014-12-31\'))' # Time Series
    exec(str)
    str=f'ds_{fb[ifb]}_moc = ds_{fb[ifb]}_moc.resample(time=\'1Y\', closed=\'left\').mean(\'time\')' # Yearly average
    exec(str)
    str=f'moc_{fb[ifb]} = ds_{fb[ifb]}_moc[\'MOC\']' # Variable
    exec(str)
    str=f'del ds_{fb[ifb]}_moc'
    exec(str)

In [ ]:
%%time
for ifb in range(0,2): 
    print(f"Forcing_variant: {ifb}")
    str=f'moc_{fb[ifb]}[\'moc_z\'] = -0.01*moc_{fb[ifb]}.moc_z' # cm to m and negative depth
    print(str)
    exec(str) 
    str=f'zz_{fb[ifb]}, yy_{fb[ifb]} = np.meshgrid(moc_{fb[ifb]}.moc_z, moc_{fb[ifb]}.lat_aux_grid)'
    print(str)
    exec(str) 
    str=f'zz_{fb[ifb]}=(np.array([zz_{fb[ifb]}]*len(moc_{fb[ifb]}[\'member_id\']))).transpose(0,2,1)'
    print(str)
    exec(str)
    str=f'yy_{fb[ifb]}=(np.array([yy_{fb[ifb]}]*len(moc_{fb[ifb]}[\'member_id\']))).transpose(0,2,1)'
    print(str)
    exec(str)

In [ ]:
matrix = np.empty((len(moc_smbb['time']),len(moc_smbb['member_id']),2,3)) * np.nan

In [ ]:
%%time
for ifb in range(0,2): # forcing variant (cmip6=0, smbb=1)
    print(f"Forcing_variant: {ifb}")
    for it in range(0,len(moc_smbb['time'])): # time 
        print(f"Time: {it}")
        
        # Maximum AMOC for the North Atlantic
        str=f'''amoc = moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0).where(
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['moc_z'] > -3500) & 
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['lat_aux_grid'] > 0) &
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['lat_aux_grid'] < 70),
        drop=True
        )'''
        exec(str)
        str=f'matrix[it,:,ifb,0]=amoc.max(dim=[\'moc_z\',\'lat_aux_grid\'])' 
        exec(str)
        
        # Maximum AMOC for the South Atlantic
        str=f'''amoc = moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0).where(
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['moc_z'] > -3500) & 
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['lat_aux_grid'] > -70) &
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['lat_aux_grid'] < 0),
        drop=True
        )'''
        exec(str)
        str=f'matrix[it,:,ifb,1]=amoc.max(dim=[\'moc_z\',\'lat_aux_grid\'])' 
        exec(str)
        
        # Minimum AMOC for the South Atlantic
        str=f'''amoc = moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0).where(
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['moc_z'] > -3500) & 
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['lat_aux_grid'] > -70) &
        (moc_{fb[ifb]}.isel(time=it,transport_reg=1,moc_comp=0)['lat_aux_grid'] < 0),
        drop=True
        )'''
        exec(str)
        str=f'matrix[it,:,ifb,2]=amoc.min(dim=[\'moc_z\',\'lat_aux_grid\'])' 
        exec(str)
        del amoc

In [ ]:
matrix_moc = xr.Dataset({"MOC": (("time","member_id","forcing_variant","basin"), matrix)},
                 coords={
                     "time": moc_cmip6.coords['time'], 
                     "member_id": moc_cmip6.coords['member_id'],
                     "forcing_variant": ['cmip6 (0)','smbb (1)'],
                     "basin": ['North Atlantic Max (0)','South Atlantic Max (1)','South Atlantic Min (2)'],
                     "transport_reg": [1],
                     "moc_comp": [0],
                     "experiment": ['historical'],
                 },
                )

In [ ]:
path = '/glade/scratch/mauricio/LENS2_south_atlantic/MOC/'.format(getpass.getuser())
matrix_moc.to_netcdf(path+'moc_tseries.nc')

In [ ]:
tempo=xr.cftime_range(start='1850', end='2015',periods=None, freq='A',normalize=False, name=None, closed=None, calendar='standard')

In [ ]:
plt.close()
fig = plt.figure(figsize=(14,6))
ax = plt.axes()
#ax.plot(tempo, matrix_moc.MOC.isel(basin=1, forcing_variant=0), 'r')
ax.plot(tempo, matrix_moc.MOC.isel(basin=0,forcing_variant=1), label='_Hidden', color='blue', linewidth=1, alpha=0.1)
ax.plot(tempo, matrix_moc.MOC.isel(basin=0,forcing_variant=0),label='_Hidden', color='red', linewidth=1, alpha=0.1)
ax.plot(tempo, matrix_moc.MOC.isel(basin=0,forcing_variant=0).mean('member_id'), label='smbb', color='red', linewidth=3)
ax.plot(tempo, matrix_moc.MOC.isel(basin=0,forcing_variant=1).mean('member_id'), label='cmip6', color='blue', linewidth=3)
plt.title("AMOC (North Atlantic)")
plt.xlabel("Years")
plt.ylabel("Volume Transport [Sv]")
plt.legend()

In [ ]:
plt.close()
fig = plt.figure(figsize=(14,6))
ax = plt.axes()
#ax.plot(tempo, matrix_moc.MOC.isel(basin=1, forcing_variant=0), 'r')
ax.plot(tempo, matrix_moc.MOC.isel(basin=1,forcing_variant=1), label='_Hidden', color='blue', linewidth=1, alpha=0.1)
ax.plot(tempo, matrix_moc.MOC.isel(basin=1,forcing_variant=0),label='_Hidden', color='red', linewidth=1, alpha=0.1)
ax.plot(tempo, matrix_moc.MOC.isel(basin=1,forcing_variant=0).mean('member_id'), label='smbb', color='red', linewidth=3)
ax.plot(tempo, matrix_moc.MOC.isel(basin=1,forcing_variant=1).mean('member_id'), label='cmip6', color='blue', linewidth=3)
plt.title("AMOC (South Atlantic)")
plt.xlabel("Years")
plt.ylabel("Volume Transport [Sv]")
plt.legend()

In [ ]:
plt.close()
fig = plt.figure(figsize=(14,6))
ax = plt.axes()
#ax.plot(tempo, matrix_moc.MOC.isel(basin=1, forcing_variant=0), 'r')
ax.plot(tempo, matrix_moc.MOC.isel(basin=2,forcing_variant=1), label='_Hidden', color='blue', linewidth=1, alpha=0.1)
ax.plot(tempo, matrix_moc.MOC.isel(basin=2,forcing_variant=0),label='_Hidden', color='red', linewidth=1, alpha=0.1)
ax.plot(tempo, matrix_moc.MOC.isel(basin=2,forcing_variant=0).mean('member_id'), label='smbb', color='red', linewidth=3)
ax.plot(tempo, matrix_moc.MOC.isel(basin=2,forcing_variant=1).mean('member_id'), label='cmip6', color='blue', linewidth=3)
plt.title("AMOC (South Atlantic)")
plt.xlabel("Years")
plt.ylabel("Volume Transport [Sv]")
plt.legend()